In [1]:
import os
import sys
import scipy
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

os.chdir("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/")
sys.path.append("/Users/kipnisal/DS/BiblicalScripts/bib-scripts/src")

params = catalog.load('parameters')

data = catalog.load("data_proc")
vocab = catalog.load("vocabulary")

known_authors = params['known_authors']

2022-03-27 09:35:22,517 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...
2022-03-27 09:35:22,517 - kedro.io.data_catalog - INFO - Loading data from `data_proc` (CSVDataSet)...
2022-03-27 09:35:22,685 - kedro.io.data_catalog - INFO - Loading data from `vocabulary` (CSVDataSet)...


## Issues: 
- with $c=2$, expect to get the same result either with one_vs_many or div_persuit. This is related to the fact that in 'diversity_persuit' the P-values are not uniform under the null -- check this!!
- Looks like this is becasue we evaluate $SS_{fit}$

In [2]:
import numpy as np
from scipy.stats import f as fdist
from scipy.stats import t as tdist
from scipy.stats import norm
from scipy.stats import ttest_ind as ttest
            
from twosample import bin_allocation_test
from multitest import MultiTest
from typing import List

from sklearn.metrics import pairwise_distances

In [132]:
class CentroidSimilarity(object):
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self._global_mean = np.mean(X, 0)
        self._global_std = np.std(X, 0)
        
        self._cls_mean = np.zeros((len(self.classes_),X.shape[1]))
        self._cls_std = np.zeros((len(self.classes_),X.shape[1]))
        self._cls_n = np.zeros((len(self.classes_),X.shape[1]))
        
        for i,_ in enumerate(self.classes_):
            X_cls = X[y == self.classes_[i]]
            self._cls_mean[i] = np.mean(X_cls, 0)
            self._cls_std[i] = np.std(X_cls, 0)
            self._cls_n[i] = len(X_cls)
        self._mat = (self._cls_mean.T / np.linalg.norm(self._cls_mean, axis=1)).T
        self._mask = np.ones_like(self._mat)

        
    def prob_func(self, response):
        return np.exp(response) / (1 + np.exp(response))
    
    def get_centroids(self):
        return self._mat * self._mask
        
    def predict_log_proba(self, X):
        response = X @ self.get_centroids().T
        return self.prob_func(response)
    
    def predict(self, X):
        probs = self.predict_log_proba(X)
        return np.argmax(probs, 1)

class CentroidSimilarityFeatureSelection(CentroidSimilarity):
    
    def fit(self, X, y, method='one_vs_all'):
        
        super().fit(X, y)
        self._cls_response = np.zeros(len(self.classes_))
        
        for i, cls in enumerate(self.classes_):
            self._mask[i] = self.get_cls_mask(i, method=method)
        
    
    def get_pvals(self, cls_id, method='one_vs_all'):
        """
        compute P-values associated with each feature
        for the given class
        """
        
        mu1 = self._cls_mean[cls_id]
        n1 = self._cls_n[cls_id]
        std1 = self._cls_std[cls_id]
        nG = self._cls_n.sum(0)
        stdG = self._global_std
        muG = self._global_mean

        assert(method in ['one_vs_all', 'diversity_persuit'])
        if method == 'one_vs_all' :
            pvals,_,_ = one_vs_all_ANOVA(n1, nG, mu1, muG, std1, stdG)
        if method == 'diversity_persuit':
            pvals,_,_ = diversity_persuit_ANOVA(self._cls_n,
                                                self._cls_mean,
                                                self._cls_std)
        return pvals

    
    def get_cls_mask(self, cls_id, method='one_vs_all'):
        """
        compute class feature mask
        """        
        pvals = self.get_pvals(cls_id, method=method)
        
        mt = MultiTest(pvals)
        hc, hct = mt.hc_star(gamma=.2)
        self._cls_response[cls_id] = hc
        mask = pvals < hct
        
        return mask
        
def diversity_persuit_ANOVA(nn, mm, ss):
    """
    F-test for discoverying discriminating features
    
    The test is vectorized along the last dimention where
    different entires corresponds to different features
    
    Args:
    -----
    :nn:  vector indicating the number of elements in each class
    :mm:  matrix of means; the (i,j) entry is the mean response of
          class i in feature j
    :ss:  matrix of standard errors; the (i,j) entry is the standard
          error of class i in feature j
    
    """
    muG = np.sum(mm * nn, 0) / np.sum(nn, 0) # global mean
    SSfit = np.sum(nn * (mm - muG) ** 2, 0)  
    SStot = np.sum(nn * (ss ** 2), 0)
    SSerr = np.sum(nn * (ss ** 2), 0)
    #SSerr = SStot - SSfit

    dfn = len(nn) - 1
    dfd = np.sum(nn, 0) - len(nn)

    F = ( SSfit / dfn ) / ( SSerr / dfd )
    return fdist.sf(F, dfn, dfd), SSerr, SSfit        
        
        
def one_vs_all_ANOVA(n1, nG, mu1, muG, std1, stdG):
    n2 = nG - n1
    mu2 = (muG * nG - mu1 * n1) / (nG - n1)
    SSfit = n1 * (mu1 - muG) ** 2 + n2 * (mu2 - muG) ** 2
    #SSerr = std1 * n1 + std2 * n2 we don't know std2
    SStot = stdG ** 2 * nG
    SSerr = SStot - SSfit

    F = ( SSfit / 1 ) / ( SSerr / (nG - 2) )
    return fdist.sf(F, 1, nG - 2), SSerr, SSfit        
        
def two_sample_t_test(mean1, mean2, std1, std2, n1, n2):
    assert n1 + n2 > 2
    sp = np.sqrt( (std1 * (n1 - 1) + std2 * (n2 - 1)) / (n1 + n2 - 2)) 
    t = (mean1 - mean2) / sp
    return tdist.sf(np.abs(t))


In [133]:
def eval_accuracy(clf, X, y):
    y_pred = clf.predict(X)
    return np.mean(y_pred == y)

In [254]:
def atomic_exp(p, n, c, sig, mu, eps, test_frac):
    
    Z = np.random.randn(n, p)
    y = np.random.randint(c, size=n)

    mus = np.zeros((c, p))
    for i in range(c):
        idcs = np.random.rand(p) < eps
        mus[i][idcs] = mu * (1-2*(np.random.rand(np.sum(idcs)) > .5)) / 2

    true_mask = mus != 0

    X = mus[y] + sig * Z

    train_split_mask = np.random.rand(len(X)) > test_frac

    X_train = X[train_split_mask]
    y_train = y[train_split_mask]

    X_test = X[~train_split_mask]
    y_test = y[~train_split_mask]

    dist_mat = pairwise_distances(mus)
    delta_mean = np.sum(dist_mat) / (c * (c - 1))
    delta_min = np.min(dist_mat + 1e9 * np.eye(len(dist_mat)))
    
    cs = CentroidSimilarity()
    cs.fit(X_train, y_train)
    acc = eval_accuracy(cs, X_test, y_test)

    csf1 = CentroidSimilarityFeatureSelection()
    csf1.fit(X_train, y_train, method='one_vs_all')
    pvals1 = csf1.get_pvals(cls_id = 0, method='one_vs_all')
    acc1 = eval_accuracy(csf1, X_test, y_test)

    csf2 = CentroidSimilarityFeatureSelection()
    csf2.fit(X_train, y_train, method='diversity_persuit')
    pvals2 = csf2.get_pvals(cls_id = 0, method='diversity_persuit')
    acc2 = eval_accuracy(csf2, X_test, y_test)

    def get_FDR(csf, true_mask):
        FP = np.sum(csf._mask.any(0)[~true_mask.any(0)])
        TP = np.sum(csf._mask.any(0)[true_mask.any(0)])
        return FP / (FP + TP)

    hc1 = MultiTest(pvals1).hc()[0]
    hc2 = MultiTest(pvals2).hc()[0]

    return dict({'acc': acc, 
           'acc1': acc1, 'acc2': acc2,
            'hc1': hc1, 'hc2': hc2,
            'fdr1' : get_FDR(csf1, true_mask), 
            'fdr2' : get_FDR(csf2, true_mask),
            'eps': eps,
            'n' : n, 'p' : p, 'mu': mu, 'c': c, 
            'delta_mean': delta_mean,
            'delta_min': delta_min,
            'mask_sum' : np.sum(true_mask),
           })

In [249]:
beta = .6
((1 - np.sqrt(1-beta)) ** 2) * (beta >= .75) + (beta/2) * (beta < .75)

0.3

In [265]:
p = 50000
n = int(2 * np.log(p) ** 2)
c = 5
sig = 1
r = 0.4 / c
beta = .7

test_frac = .2
mu = np.sqrt(r*np.log(p))
eps = p ** (-beta)

nMonte = 10
df = pd.DataFrame()
for itr in tqdm(range(nMonte)):
    r = atomic_exp(p, n, c, sig, mu, eps, test_frac)
    r['itr'] = itr
    df = df.append(r, ignore_index=True)
    
df.mean()

100%|██████████| 10/10 [00:06<00:00,  1.43it/s]


acc               0.221331
acc1              0.272958
acc2              0.273287
c                 5.000000
delta_mean        3.378386
delta_min         3.082302
eps               0.000514
fdr1              0.985969
fdr2              0.896540
hc1               2.035384
hc2               2.795632
itr               4.500000
mask_sum        132.700000
mu                0.930367
n               234.000000
p             50000.000000
dtype: float64

In [241]:
Z = np.random.randn(n, p)
y = np.random.randint(c, size=n)

mus = np.zeros((c, p))
for i in range(c):
    idcs = np.random.rand(p) < eps
    mus[i][idcs] = mu * (1-2*(np.random.rand(np.sum(idcs)) > .5)) / 2

true_mask = mus != 0

X = mus[y] + sig * Z

train_split_mask = np.random.rand(len(X)) > test_frac

X_train = X[train_split_mask]
y_train = y[train_split_mask]

X_test = X[~train_split_mask]
y_test = y[~train_split_mask]

dist_mat = pairwise_distances(mus)
delta_mean = np.sum(dist_mat) / (c * (c - 1))
delta_min = np.min(dist_mat + 1e9 * np.eye(len(dist_mat)))

cs = CentroidSimilarity()
cs.fit(X_train, y_train)
acc = eval_accuracy(cs, X_test, y_test)

csf1 = CentroidSimilarityFeatureSelection()
csf1.fit(X_train, y_train, method='one_vs_all')
pvals1 = csf1.get_pvals(cls_id = 0, method='one_vs_all')
acc1 = eval_accuracy(csf1, X_test, y_test)

csf2 = CentroidSimilarityFeatureSelection()
csf2.fit(X_train, y_train, method='diversity_persuit')
pvals2 = csf2.get_pvals(cls_id = 0, method='diversity_persuit')
acc2 = eval_accuracy(csf2, X_test, y_test)

def get_FDR(csf, true_mask):
    #FP = np.sum(csf._mask > true_mask)
    #TP = np.sum(csf._mask*true_mask)
    FP = np.sum(csf._mask.any(0)[~true_mask.any(0)])
    TP = np.sum(csf._mask.any(0)[true_mask.any(0)])
    
    return FP / (FP + TP)

hc1 = MultiTest(pvals1).hc()[0]
hc2 = MultiTest(pvals2).hc()[0]

dict({'acc': acc, 
       'acc1': acc1, 'acc2': acc2,
        'hc1': hc1, 'hc2': hc2,
        'fdr1' : get_FDR(csf1, true_mask), 
        'fdr2' : get_FDR(csf2, true_mask),
        'eps': eps,
        'n' : n, 'p' : p, 'mu': mu, 'c': c, 
        'delta_mean': delta_mean,
        'delta_min': delta_min,
        'mask_sum' : np.sum(true_mask),
       })

{'acc': 0.375,
 'acc1': 0.625,
 'acc2': 0.625,
 'hc1': 1.7139266906616784,
 'hc2': 1.7139266906615778,
 'fdr1': 0.972027972027972,
 'fdr2': 0.972027972027972,
 'eps': 9.999999999999995e-05,
 'n': 46,
 'p': 100000,
 'mu': 2.628260884878466,
 'c': 2,
 'delta_mean': 4.917025877137264,
 'delta_min': 4.917025877137264,
 'mask_sum': 14}

In [245]:
FP = np.sum(csf1._mask.any(0)[~true_mask.any(0)])
TP = np.sum(csf1._mask.any(0)[true_mask.any(0)])
FP / (FP + TP)

0.972027972027972

In [246]:
csf1._mask.any(0)[true_mask.any(0)].sum()

12

## Bible Data

In [424]:
from biblical_scripts.pipelines.sim.nodes import (
    build_model, model_predict, _prepare_data)
from sklearn.model_selection import KFold

In [425]:
data = _prepare_data(data[data.to_report])
data_known = data[data.author.isin(known_authors)]
lo_docs = data_known.doc_id.unique()

2022-03-24 11:18:18,238 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [426]:
df_res = pd.DataFrame()
for doc in tqdm(lo_docs): # leave-one-out
    ds1 = data_known[data_known.doc_id == doc]
    data_train = data_known.drop(ds1.index) # leave-one-out
    md, vocab = build_model(data_train, vocab, params['model'])
    om = OneVsMany()
    om.fit_MultiDoc(md)
    om.fit_class_means(md, data_train)
    r = om.predict_proba(ds1)
    #r = om.predict_response(ds1)
    res = pd.DataFrame(r, index = ['avg']).T.reset_index().rename(columns={'index' : 'cls'})
    res['doc_tested'] = doc
    res['len'] = len(ds1)
    df_res = df_res.append(res, ignore_index=True)

  0%|          | 0/49 [00:00<?, ?it/s]

2022-03-24 11:18:18,376 - root - INFO - Building CompareDocs model using 48 documents. 


  0%|          | 0/49 [00:00<?, ?it/s]


NameError: name 'OneVsMany' is not defined

In [126]:
df_res.loc[: ,'true_class'] = df_res.doc_tested.apply(lambda x : x.split('|')[0])
#df_res.loc[:, 'neg_raw'] = -df_res['raw']

for value in df_res.columns[df_res.columns.str.contains(r'pval|neg_raw|avg')]:        
    idx = df_res.groupby('doc_tested')[value].idxmax(axis=1)
    acc = np.mean(df_res.loc[idx, 'cls'] == df_res.loc[idx, 'true_class'])
    print(f"Accuracy with {value} = {acc}")
    #df_res['predicted_class'] = df_res[['Dtr', 'DtrH', 'P']].idxmin(axis=1)
    #print("Accuracy: ", np.mean(df_res.predicted_class == df_res.true_class))

Accuracy with pval_2smp = 0.5714285714285714
Accuracy with pval_bartlett = 0.5306122448979592
Accuracy with pval_f = 0.6326530612244898
Accuracy with pval_levene = 0.3673469387755102
Accuracy with pval_t = 0.8367346938775511
Accuracy with pval_tLDA = 0.8571428571428571
Accuracy with pval_wilcox = 0.4897959183673469
